### 1. 데이터 생성하기
(1) ./datasets 아래에 .hwp 파일을 모두 load하기

(2) text_splitter로 분할하기

(3) all_docs에 모든 chunk들 append하기

(4) Pineone Vector database에 업로드하기

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_teddynote.document_loaders.hwp import HWPLoader
import os

document_list = []

# text splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
)

# datasets 폴더를 시작점으로 하위 폴더까지 재귀적으로 순회
for root, dirs, files in os.walk("./datasets"):
    for file in files:
        if file.endswith(".hwp"):
            file_path = os.path.join(root, file)
            loader = HWPLoader(file_path)
            # 1) 문서 로드
            docs = loader.load()
            # 2) 텍스트 스플리터로 분할
            split_docs = text_splitter.split_documents(docs)
            document_list.extend(split_docs)

print(f"The number of Splitted Chunks: {len(document_list)}")

The number of Splitted Chunks: 3065


In [8]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [22]:
#Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'ds-chatbot'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [11]:
#Chroma
# from langchain_chroma import Chroma

# database=Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

#database=Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding) #이미 만들어 놓은 데베 사용

### 2. 답변 생성을 위한 Retrieval 및 Prompt 활용하기
(1) RetrievalQA에 전달하기 위해 retriever 생성

(2) LangChain에서 제공하는 rlm/rag-prompt 사용

In [4]:
query = '대학원 영어 면제 인정 기준은?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='dedc4edc-9982-46b0-94bc-4035f496f15a', metadata={'source': './datasets/2편 학칙/1. 대학원학칙시행세칙(20230710)_일부개정.hwp'}, page_content='구분(외국어) | 과정     | 시험 종류                  | 합격 기준(등급/점수)                | 비고                  ||----------|--------|----------------------|---------------------------|---------------------|| 영어      | 석·박사  | PBT(시험지형)            | 500점 이상                  | 일반대학원 공통사항     ||          |        | CBT(컴퓨터형)            | 173점 이상                  |                     ||          |        | IBT(인터넷형)            | 61점 이상                   |                     ||          |        | TOEIC                  | 700점 이상                  |                     ||          |        | TEPS(서울대형)           | 600점 이상                  |'),
 Document(id='769b609e-e98e-4b79-bdea-88e905a58192', metadata={'source': './datasets/2편 학칙/1. 대학원학칙시행세칙(20230710)_일부개정.hwp'}, page_content='||          |        | TEPS(서울대형)  

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

### 3. 답변 생성하기
(1) RetrievalQA를 통해 LLM에 전달

In [6]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '대학원 영어 면제 인정 기준은?',
 'result': '대학원 영어 면제 인정 기준은 TOEFL PBT 500점 이상, CBT 173점 이상, IBT 61점 이상, TOEIC 700점 이상, TEPS 600점 이상 또는 New TEPS 326점 이상입니다.'}